In [ ]:
from pennystockpipeline.PennyStockData import PennyStockData
import numpy as np
import pandas as pd
from time import gmtime, strftime
from datetime import datetime, timedelta
import torch

torch.manual_seed(0)

## Initializing
DATABASE_NAME_WITH_PATH = "mod_historicaldata.db"
TABLE_NAME = "all_historical_modified"

#psd = PennyStockData(database_name_with_path, table_name, impute=True, verbose=2).get_columns(columns).normalize(['close', 'volume_weighted_average']).create_sequences(sequence_length, prediction_length)
psd = PennyStockData(DATABASE_NAME_WITH_PATH, TABLE_NAME, impute=True, verbose=2)


In [ ]:
#columns = ['ticker_id', 'p_date', 'close', 'volume_weighted_average']
COLUMNS = ['ticker_id', 'p_date', 'p_time', 'volume_weighted_average']
COLUMNS_TO_NORMALIZE = ['volume_weighted_average']

SEQUENCE_LENGTH = 20
PREDICTION_LENGTH = 20

TRAIN_TEST_SPLIT = 0.8
psd = psd.get_columns(COLUMNS).normalize(COLUMNS_TO_NORMALIZE).create_sequences(SEQUENCE_LENGTH, PREDICTION_LENGTH).split_dataset(split=TRAIN_TEST_SPLIT, to_torch=True)


In [ ]:
#model = None

In [ ]:
# Library for implementing Deep Neural Network 
from pennystockpipeline.PennyStockModel import PennyStockModel

# Prepare model parameters
INPUT_SIZE = len(COLUMNS_TO_NORMALIZE)
NUM_LAYERS = 2
HIDDEN_SIZE = 64
OUTPUT_SIZE = len(COLUMNS_TO_NORMALIZE)
DROPOUT = 0
# Define the model, loss function, and optimizer
model = PennyStockModel(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, OUTPUT_SIZE, DROPOUT, device='cuda')
model = model.to(model.device)
loss_fn = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
print(model)


In [ ]:
##
## single ticker, ticker_id = 9
##

#torch.random_seed(0), 
#batch = 32								batch = 16
#model	epoch=40						model	epoch=40
#best_loss: 0.0020826306399612804		best_loss: 0.004081749377291999
#diff	0.0005							diff	0.0003

#torch.random_seed(0), 
#batch = 32								batch = 16
#model	epoch=50						model	epoch=50
#best_loss: 0.002037114063568879		best_loss: 0.004062295620315126
#diff	-0.0003							diff	0.0034

##
## multiple ticker, ticker_id <> 30
##

#torch.random_seed(0), 
#batch = 32								batch = 16
#model	epoch=40						model	epoch=40
#best_loss: 0.060114274077022856		best_loss: 0.11994675780084663
#diff	0.0008							diff	-0.0021

#torch.random_seed(0), 
#batch = 32								batch = 16
#model	epoch=50						model	epoch=50
#best_loss: 0.060114274077022856		best_loss: 0.11994675780084663 
#diff	-0.005							diff	-0.0032
 
## can we use transfer learning?

In [ ]:
BATCH_SIZE = 32   # batch 16, run epoch 40; batch 32, run epoch 40 or 50 with torch.random_seed(0)
model = model.create_dataloaders(psd, BATCH_SIZE)

In [ ]:
NUM_EPOCHS = 40
# Train the model
#model = model.load_model()

model = model.train_model(loss_fn, optimizer, num_epochs = NUM_EPOCHS).plot_training_test_loss()#.save_model()


In [ ]:
FORECAST_STEPS = 20
model = model.forecast(num_forecast_steps = FORECAST_STEPS)


In [ ]:
#model = model.load_model()
model = model.plot_forecasting()#, i_sequence_to_plot, i_forecasted_values, i_combined_index)


In [ ]:
#model = model.save_model()

In [ ]:
'''
# Library for implementing Deep Neural Network 
psd2 = psd

from pennystockpipeline.PennyStockModel2 import PennyStockModel2

# Prepare model parameters
INPUT_SIZE = len(COLUMNS_TO_NORMALIZE)
NUM_LAYERS = 2
HIDDEN_SIZE = 32
OUTPUT_SIZE = len(COLUMNS_TO_NORMALIZE)
DROPOUT = 0.2

# Define the model, loss function, and optimizer
model2 = PennyStockModel2(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, OUTPUT_SIZE, DROPOUT, device='cuda')
model2 = model2.to(model2.device)
loss_fn2 = torch.nn.MSELoss(reduction='mean')
optimizer2 = torch.optim.Adam(model2.parameters(), lr=1e-3)
print(model2)

BATCH_SIZE = 64
model2 = model2.create_dataloaders(psd2, BATCH_SIZE)
NUM_EPOCHS = 100
# Train the model
#model2 = model2.load_model()
model2 = model2.train_model(loss_fn2, optimizer2, num_epochs = NUM_EPOCHS).plot_training_test_loss()#.save_model()

model2 = model2.forecast(num_forecast_steps = FORECAST_STEPS)
model2 = model2.plot_forecasting()#, i_sequence_to_plot, i_forecasted_values, i_combined_index)
'''